In [1]:
import requests
import json
import time
from datetime import datetime
from datetime import timedelta

import sys
import pandas as pd

In [2]:
def today_eqkmsg(npage):
        params = {'ServiceKey':mykey, 'fromTmFc':eqkt, 'toTmFc':eqkt,
                'dataType':datatype, 'pageNo':npage, 'numOfRows':10}
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        res = json.loads(r.text).get('response')
        return res

In [3]:
def eqkmsg_count(fromt=None, tot=None, npage=1, nrow=10, gettoday=False, recent_3days=False):
    global rescount, rcode, res

    if type(fromt) == int:
        fromt = str(fromt)
    if type(tot) == int:
        tot = str(tot)
    if (type(fromt)!=None) and (tot == None):
        tot = eqkt

    if recent_3days:
        ago_3day = str(datetime.today() - timedelta(days=3))[0:10].split('-')
        fromt = ago_3day[0]+ago_3day[1]+ago_3day[2]
        print('최근 3일 동안 발생한 지진통보를 검색합니다.')
        params = {'ServiceKey':mykey, 'fromTmFc':fromt, 'toTmFc':tot,
            'dataType':datatype, 'pageNo':npage, 'numOfRows':nrow}
    
    if gettoday:
        fromt, tot = eqkt, eqkt
        print('오늘의 지진 통보를 검색합니다.')
        params = {'ServiceKey':mykey, 'fromTmFc':fromt, 'toTmFc':tot,
            'dataType':datatype, 'pageNo':npage, 'numOfRows':nrow}
    else:
        print('조회기간 {}년 {}월 {}일 ~ {}월 {}일'.format(fromt[0:4],fromt[4:6],fromt[6:8],tot[4:6],tot[6:8]))
        params = {'ServiceKey':mykey, 'fromTmFc':fromt, 'toTmFc':tot,
            'dataType':datatype, 'pageNo':npage, 'numOfRows':nrow}
    try:
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
    except:
        print('기상청 api 호출 실패, 제대로 입력했는지 확인해주세요')
        sys.exit(1)

    if r.status_code == 200:
        try:
            res = json.loads(r.text).get('response')
            rcode = res.get('header')['resultCode']
            print('기상청 api 호출 성공 (API result code:',rcode+')')
        except:
            print('통보문 데이터를 불러오는데 실패했습니다.')
            sys.exit(1)

        if rcode == '00':
            rescount = res.get('body')['totalCount']
            if gettoday:
                print('오늘은 지진 통보가',rescount,'개 있습니다.')
            else:
                print('조회기간 동안 지진 통보는 총',rescount,'개 입니다.')
        elif rcode == '03':
            rescount = 0
            if gettoday:
                print('오늘은 지진 통보가 없습니다.')
            else:
                print('조회기간 동안 지진 통보가 없습니다.')
        elif rcode == '99':
            print('지정가능한 기간 범위를 초과했습니다 (최대 3일 조회가능)')
            sys.exit(1)

    if r.status_code != 200:
        print('기상청 api 호출 실패 code:',r.status_code)
        sys.exit(1)

    return rescount

In [4]:
def eqk_data(today=True):
    global eqk_list
    eqk_list = []
    eqk_dict = {}
    rpages = (rescount//10)+1
    if today:
        if rcode == '00':
            for i in range(0, rpages):
                try:
                    r = today_eqkmsg(str(i+1))
                    raw = r.get('body').get('items')['item']
                except:
                        print('지진 통보문 목록 불러오기 실패')
                        sys.exit(1)
                for j in range(0, len(raw)):
                    data = raw[j]
                    eqk_dict = {}
                    tmEqktxt = str(data.get('tmEqk'))
                    eqk_dict['id'] = eqkt+'-'+str(i)+str(j)
                    eqk_dict['day'] = tmEqktxt[0:8]
                    eqk_dict['eqkt'] = tmEqktxt[8:14]
                    eqk_dict['fct'] = str(data.get('tmFc'))[8:12]
                    eqk_dict['mt'] = data.get('mt')
                    eqk_dict['loc'] = data.get('loc')
                    eqk_dict['lat'] = data.get('lat')
                    eqk_dict['lon'] = data.get('lon')
                    eqk_dict['mapURL'] = data.get('img')
                    eqk_dict['fctype'] = data.get('fcTp')
                    eqk_dict['weekday'] = datetime.today().strftime('%A')
                    eqk_list.append(eqk_dict)
                time.sleep(1)
            pdcol = eqk_list[0].keys()
            eqk_df = pd.DataFrame(columns=pdcol, data=[eqk_list[i].values() for i in range(0,len(eqk_list))])
    else:
        if rcode == '00':
            for i in range(0, rpages):
                try:
                    r = res
                    raw = r.get('body').get('items')['item']
                except:
                        print('지진 통보문 목록 불러오기 실패')
                        sys.exit(1)
                for j in range(0, len(raw)):
                    data = raw[j]
                    eqk_dict = {}
                    tmEqktxt = str(data.get('tmEqk'))
                    year = tmEqktxt[0:4]
                    month = tmEqktxt[4:6]
                    day = tmEqktxt[6:8]
                    eqk_dict['id'] = tmEqktxt[0:8]+'-'+str(i)+str(j)
                    eqk_dict['day'] = tmEqktxt[0:8]
                    eqk_dict['eqkt'] = tmEqktxt[8:14]
                    eqk_dict['fct'] = str(data.get('tmFc'))[0:12]
                    eqk_dict['mt'] = data.get('mt')
                    eqk_dict['loc'] = data.get('loc')
                    eqk_dict['lat'] = data.get('lat')
                    eqk_dict['lon'] = data.get('lon')
                    eqk_dict['mapURL'] = data.get('img')
                    eqk_dict['fctype'] = data.get('fcTp')
                    eqk_dict['weekday'] = datetime(int(year),int(month),int(day)).strftime('%A')
                    eqk_list.append(eqk_dict)
                time.sleep(1)
            pdcol = eqk_list[0].keys()
            eqk_df = pd.DataFrame(columns=pdcol, data=[eqk_list[i].values() for i in range(0,len(eqk_list))])
    return eqk_df

In [5]:
def main():
    global mykey, endpoint, datatype, eqkt
    mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
    endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
    datatype = 'JSON'
    today = str(datetime.today())[0:10].split('-')
    eqkt = today[0]+today[1]+today[2]
    eqkmsg_count(recent_3days=True)
    if rcode != '00':
        print('end')
    else:
        eqk_df = eqk_data(today=False)
    print(eqk_df)

In [6]:
if __name__ == '__main__':
    main()

최근 3일 동안 발생한 지진통보를 검색합니다.
조회기간 2022년 04월 16일 ~ 04월 19일
기상청 api 호출 성공 (API result code: 00)
조회기간 동안 지진 통보는 총 6 개 입니다.
            id       day    eqkt           fct   mt  \
0  20220419-00  20220419  133341  202204191405  6.0   
1  20220419-01  20220419  102310  202204191055  6.2   
2  20220419-02  20220419  081600  202204190823  5.3   
3  20220418-03  20220418  142238  202204181426  2.3   
4  20220417-04  20220417  233329  202204172335  2.3   
5  20220416-05  20220416  020834  202204160212  2.6   

                          loc    lat     lon  \
0     통가 통가타푸섬 동남동쪽 130km 해역  -21.45 -173.92   
1         필리핀 마티 동북동쪽 65km 해역   7.12  126.78   
2  일본 이바라키현(혼슈) 미토 북쪽 60km 지역  36.90  140.40   
3         인천 옹진군 서남서쪽 52km 지역  37.26  126.10   
4      경남 창원시 마산회원구 동쪽 2km 지역  35.22  128.60   
5           전남 고흥군 북쪽 16km 지역  34.75  127.26   

                                              mapURL  fctype   weekday  
0  http://www.weather.go.kr/repositary/image/eqk/...       2   Tuesday  
1  http://www.